In [1]:
import json
from openai import OpenAI
from groq import Groq
import os
from sentence_transformers import SentenceTransformer
from tqdm.auto import tqdm
import pickle
from elasticsearch import Elasticsearch
import pandas as pd
import requests
import numpy as np
from concurrent.futures import ThreadPoolExecutor
from requests.exceptions import HTTPError

import time

/home/codespace/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
prompt1_template = """
You are an expert evaluator for a Retrieval-Augmented Generation (RAG) system.
Your task is to analyze the relevance of the generated answer compared to the original answer provided.
Based on the relevance and similarity of the generated answer to the original answer, you will classify
it as "NON_RELEVANT", "PARTLY_RELEVANT", or "RELEVANT".

Here is the data for evaluation:

Original Answer: {answer_orig}
Generated Question: {question}
Generated Answer: {answer_llm}

Please analyze the content and context of the generated answer in relation to the original
answer and provide your evaluation in parsable JSON without using code blocks:

{{
  "Relevance": "NON_RELEVANT" | "PARTLY_RELEVANT" | "RELEVANT",
  "Explanation": "[Provide a brief explanation for your evaluation]"
}}
""".strip()

prompt2_template = """
You are an expert evaluator for a Retrieval-Augmented Generation (RAG) system.
Your task is to analyze the relevance of the generated answer to the given question.
Based on the relevance of the generated answer, you will classify it
as "NON_RELEVANT", "PARTLY_RELEVANT", or "RELEVANT".

Here is the data for evaluation:

Question: {question}
Generated Answer: {answer_llm}

Please analyze the content and context of the generated answer in relation to the question
and provide your evaluation in parsable JSON without using code blocks:

{{
  "Relevance": "NON_RELEVANT" | "PARTLY_RELEVANT" | "RELEVANT",
  "Explanation": "[Provide a brief explanation for your evaluation]"
}}
""".strip()

In [4]:
client =  Groq(api_key = os.environ['GROQ_API_KEY2'])
def llm(prompt, model = 'mixtral-8x7b-32768'):
    retries = 5
    for i in range(retries):
        try:
            response = client.chat.completions.create(
                model= 'llama3-8b-8192',
                messages=[{"role": "user", "content": prompt}]
            )
            json_response = response.choices[0].message.content
            return json_response
        except HTTPError as e:
            if e.response.status_code == 429:  # Rate limit error
                retry_after = float(e.response.json()['error']['message'].split('in ')[-1].split('s')[0])
                time.sleep(retry_after)
            else:
                raise
        except Exception as e:
            if i < retries - 1:
                time.sleep(2 ** i)  # Exponential backoff
            else:
                raise

In [6]:
for i in range(1, 83):
    evaluations = []
    results = []
    json_evaluations = []
    
    # Load the pickle file
    with open(f"../data/vietnamese_rag/llm_answer/llm_answer{i}.pkl", 'rb') as file:
        r = pickle.load(file)
    results.extend(r)
    
    # Convert results to DataFrame
    df = pd.DataFrame(results)
    samples = df.to_dict(orient='records')
    
    # Generate evaluations
    for record in tqdm(samples):
        prompt = prompt2_template.format(**record)
        evaluation = llm(prompt)
        evaluations.append(evaluation)
    
    # Parse evaluations
    for j, str_eval in enumerate(evaluations):
        try:
            json_eval = json.loads(str_eval)
        except json.JSONDecodeError as e:
            print(f"JSONDecodeError: {e}")
            # Attempt to fix the JSON string
            try:
                str_eval = str_eval.rstrip('}') + '}'  # Ensure it ends with a closing brace
                json_eval = json.loads(str_eval)
            except json.JSONDecodeError as e:
                print(f"Failed to fix JSON string: {e}")
                continue  # Skip this evaluation if it cannot be fixed
        json_evaluations.append(json_eval)
    
    # Save evaluations to CSV
    if json_evaluations:
        df_evaluations = pd.DataFrame(json_evaluations)
        df_evaluations.to_csv(f'../data/vietnamese_rag/evaluations_qa/evaluations-qa{i}.csv', index=False)
        print(f"evaluations-qa{i}.csv saved")

100%|███████████████████████████████████████████████████| 15/15 [00:06<00:00,  2.44it/s]


JSONDecodeError: Expecting ',' delimiter: line 3 column 357 (char 385)
JSONDecodeError: Expecting ',' delimiter: line 3 column 415 (char 450)
JSONDecodeError: Expecting ',' delimiter: line 3 column 354 (char 382)
evaluations-qa1.csv saved


100%|███████████████████████████████████████████████████| 15/15 [00:05<00:00,  2.70it/s]


JSONDecodeError: Expecting ',' delimiter: line 3 column 300 (char 328)
JSONDecodeError: Expecting ',' delimiter: line 3 column 284 (char 319)
JSONDecodeError: Expecting ',' delimiter: line 3 column 359 (char 385)
JSONDecodeError: Expecting value: line 1 column 1 (char 0)
Failed to fix JSON string: Expecting value: line 1 column 1 (char 0)
JSONDecodeError: Expecting ',' delimiter: line 3 column 252 (char 280)
evaluations-qa2.csv saved


100%|███████████████████████████████████████████████████| 15/15 [00:39<00:00,  2.63s/it]


...

JSONDecodeError: Expecting ',' delimiter: line 3 column 237 (char 269)
JSONDecodeError: Expecting ',' delimiter: line 3 column 240 (char 266)
JSONDecodeError: Expecting ',' delimiter: line 3 column 181 (char 209)
JSONDecodeError: Expecting ',' delimiter: line 3 column 362 (char 395)
JSONDecodeError: Expecting ',' delimiter: line 3 column 238 (char 266)
JSONDecodeError: Expecting ',' delimiter: line 3 column 235 (char 263)
JSONDecodeError: Expecting ',' delimiter: line 3 column 184 (char 212)
evaluations-qa80.csv saved


100%|███████████████████████████████████████████████████| 15/15 [00:38<00:00,  2.57s/it]


JSONDecodeError: Expecting ',' delimiter: line 3 column 252 (char 284)
JSONDecodeError: Expecting value: line 1 column 1 (char 0)
Failed to fix JSON string: Expecting value: line 1 column 1 (char 0)
JSONDecodeError: Expecting ',' delimiter: line 3 column 230 (char 258)
JSONDecodeError: Expecting ',' delimiter: line 3 column 286 (char 314)
evaluations-qa81.csv saved


100%|█████████████████████████████████████████████████████| 2/2 [00:05<00:00,  2.68s/it]

JSONDecodeError: Expecting ',' delimiter: line 3 column 363 (char 398)
evaluations-qa82.csv saved


In [7]:
for i in range(83, 150):
    evaluations = []
    results = []
    json_evaluations = []
    
    # Load the pickle file
    with open(f"../data/vietnamese_rag/llm_answer/llm_answer{i}.pkl", 'rb') as file:
        r = pickle.load(file)
    results.extend(r)
    
    # Convert results to DataFrame
    df = pd.DataFrame(results)
    samples = df.to_dict(orient='records')
    
    # Generate evaluations
    for record in tqdm(samples):
        prompt = prompt2_template.format(**record)
        evaluation = llm(prompt)
        evaluations.append(evaluation)
    
    # Parse evaluations
    for j, str_eval in enumerate(evaluations):
        try:
            json_eval = json.loads(str_eval)
        except json.JSONDecodeError as e:
            print(f"JSONDecodeError: {e}")
            # Attempt to fix the JSON string
            try:
                # str_eval = "{" + str_eval.lstrip("{")  # Ensure it starts with an opening brace
                str_eval = str_eval.rstrip('}') + '}'  # Ensure it ends with a closing brace
                json_eval = json.loads(str_eval)
            except json.JSONDecodeError as e:
                print(f"Failed to fix JSON string: {e}")
                continue  # Skip this evaluation if it cannot be fixed
        json_evaluations.append(json_eval)
    
    # Save evaluations to CSV
    if json_evaluations:
        df_evaluations = pd.DataFrame(json_evaluations)
        df_evaluations.to_csv(f'../data/vietnamese_rag/evaluations_qa/evaluations-qa{i}.csv', index=False)
        print(f"evaluations-qa{i}.csv saved")

100%|███████████████████████████████████████████████████| 15/15 [00:39<00:00,  2.61s/it]


JSONDecodeError: Expecting ',' delimiter: line 3 column 215 (char 241)
JSONDecodeError: Expecting ',' delimiter: line 3 column 394 (char 420)
JSONDecodeError: Expecting ',' delimiter: line 3 column 350 (char 378)
JSONDecodeError: Expecting ',' delimiter: line 3 column 336 (char 364)
JSONDecodeError: Expecting value: line 1 column 1 (char 0)
Failed to fix JSON string: Expecting value: line 1 column 1 (char 0)
JSONDecodeError: Expecting ',' delimiter: line 3 column 256 (char 284)
evaluations-qa83.csv saved


100%|███████████████████████████████████████████████████| 15/15 [00:38<00:00,  2.57s/it]


JSONDecodeError: Expecting ',' delimiter: line 3 column 343 (char 369)
JSONDecodeError: Expecting ',' delimiter: line 3 column 364 (char 392)
JSONDecodeError: Expecting ',' delimiter: line 3 column 330 (char 358)
JSONDecodeError: Expecting ',' delimiter: line 3 column 328 (char 363)
JSONDecodeError: Expecting ',' delimiter: line 3 column 266 (char 294)
JSONDecodeError: Expecting ',' delimiter: line 3 column 374 (char 402)
JSONDecodeError: Expecting ',' delimiter: line 3 column 220 (char 246)
JSONDecodeError: Expecting ',' delimiter: line 3 column 261 (char 296)
JSONDecodeError: Expecting ',' delimiter: line 3 column 349 (char 377)
JSONDecodeError: Expecting ',' delimiter: line 3 column 169 (char 195)
evaluations-qa84.csv saved


100%|███████████████████████████████████████████████████| 15/15 [00:38<00:00,  2.55s/it]


...

JSONDecodeError: Expecting ',' delimiter: line 3 column 534 (char 569)
JSONDecodeError: Expecting ',' delimiter: line 3 column 225 (char 253)
JSONDecodeError: Expecting ',' delimiter: line 3 column 175 (char 203)
JSONDecodeError: Expecting ',' delimiter: line 3 column 447 (char 482)
evaluations-qa147.csv saved


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:38<00:00,  2.58s/it]


JSONDecodeError: Expecting ',' delimiter: line 3 column 275 (char 303)
JSONDecodeError: Expecting ',' delimiter: line 3 column 391 (char 426)
JSONDecodeError: Expecting ',' delimiter: line 3 column 284 (char 312)
JSONDecodeError: Expecting ',' delimiter: line 3 column 323 (char 351)
JSONDecodeError: Expecting ',' delimiter: line 3 column 266 (char 294)
JSONDecodeError: Expecting ',' delimiter: line 3 column 237 (char 265)
JSONDecodeError: Expecting ',' delimiter: line 3 column 290 (char 318)
JSONDecodeError: Expecting ',' delimiter: line 3 column 182 (char 210)
JSONDecodeError: Expecting ',' delimiter: line 3 column 290 (char 318)
JSONDecodeError: Expecting ',' delimiter: line 3 column 520 (char 546)
evaluations-qa148.csv saved


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:38<00:00,  2.57s/it]

JSONDecodeError: Expecting ',' delimiter: line 3 column 302 (char 328)
JSONDecodeError: Expecting ',' delimiter: line 3 column 461 (char 489)
JSONDecodeError: Expecting ',' delimiter: line 3 column 437 (char 463)
JSONDecodeError: Expecting ',' delimiter: line 3 column 317 (char 345)
JSONDecodeError: Expecting ',' delimiter: line 3 column 287 (char 315)
JSONDecodeError: Expecting value: line 1 column 1 (char 0)
Failed to fix JSON string: Expecting value: line 1 column 1 (char 0)
JSONDecodeError: Expecting ',' delimiter: line 3 column 218 (char 246)
evaluations-qa149.csv saved


In [ ]:
for i in range(150, 411):
    evaluations = []
    results = []
    json_evaluations = []
    
    # Load the pickle file
    with open(f"../data/vietnamese_rag/llm_answer/llm_answer{i}.pkl", 'rb') as file:
        r = pickle.load(file)
    results.extend(r)
    
    # Convert results to DataFrame
    df = pd.DataFrame(results)
    samples = df.to_dict(orient='records')
    
    # Generate evaluations
    for record in tqdm(samples):
        prompt = prompt2_template.format(**record)
        evaluation = llm(prompt)
        evaluations.append(evaluation)
    
    # Parse evaluations
    for j, str_eval in enumerate(evaluations):
        try:
            json_eval = json.loads(str_eval)
        except json.JSONDecodeError as e:
            print(f"JSONDecodeError: {e}")
            # Attempt to fix the JSON string
            try:
                # str_eval = "{" + str_eval.lstrip("{")  # Ensure it starts with an opening brace
                str_eval = str_eval.rstrip('}') + '}'  # Ensure it ends with a closing brace
                json_eval = json.loads(str_eval)
            except json.JSONDecodeError as e:
                print(f"Failed to fix JSON string: {e}")
                continue  # Skip this evaluation if it cannot be fixed
        json_evaluations.append(json_eval)
    
    # Save evaluations to CSV
    if json_evaluations:
        df_evaluations = pd.DataFrame(json_evaluations)
        df_evaluations.to_csv(f'../data/vietnamese_rag/evaluations_qa/evaluations-qa{i}.csv', index=False)
        print(f"evaluations-qa{i}.csv saved")

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:38<00:00,  2.57s/it]


JSONDecodeError: Expecting ',' delimiter: line 3 column 381 (char 409)
JSONDecodeError: Expecting ',' delimiter: line 3 column 304 (char 339)
Failed to fix JSON string: Expecting ',' delimiter: line 3 column 304 (char 339)
JSONDecodeError: Expecting ',' delimiter: line 3 column 421 (char 449)
JSONDecodeError: Expecting ',' delimiter: line 3 column 241 (char 269)
JSONDecodeError: Expecting value: line 1 column 1 (char 0)
Failed to fix JSON string: Expecting value: line 1 column 1 (char 0)
JSONDecodeError: Expecting ',' delimiter: line 3 column 303 (char 331)
evaluations-qa150.csv saved


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:40<00:00,  2.68s/it]


JSONDecodeError: Expecting ',' delimiter: line 3 column 342 (char 370)
JSONDecodeError: Expecting ',' delimiter: line 3 column 236 (char 264)
JSONDecodeError: Expecting ',' delimiter: line 3 column 327 (char 353)
JSONDecodeError: Expecting ',' delimiter: line 3 column 351 (char 379)
JSONDecodeError: Expecting ',' delimiter: line 3 column 398 (char 426)
JSONDecodeError: Expecting ',' delimiter: line 3 column 329 (char 355)
JSONDecodeError: Expecting ',' delimiter: line 3 column 411 (char 439)
JSONDecodeError: Expecting ',' delimiter: line 3 column 393 (char 421)
JSONDecodeError: Expecting ',' delimiter: line 3 column 190 (char 218)
JSONDecodeError: Expecting ',' delimiter: line 3 column 422 (char 457)
JSONDecodeError: Expecting ',' delimiter: line 3 column 324 (char 352)
JSONDecodeError: Expecting ',' delimiter: line 3 column 358 (char 393)
JSONDecodeError: Expecting ',' delimiter: line 3 column 220 (char 248)
JSONDecodeError: Expecting ',' delimiter: line 3 column 498 (char 533)
evalua

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:39<00:00,  2.62s/it]


...

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:39<00:00,  2.63s/it]


JSONDecodeError: Expecting value: line 1 column 1 (char 0)
Failed to fix JSON string: Expecting value: line 1 column 1 (char 0)
JSONDecodeError: Expecting value: line 1 column 1 (char 0)
Failed to fix JSON string: Expecting value: line 1 column 1 (char 0)
JSONDecodeError: Expecting ',' delimiter: line 3 column 241 (char 269)
JSONDecodeError: Expecting ',' delimiter: line 3 column 350 (char 378)
JSONDecodeError: Expecting value: line 1 column 1 (char 0)
Failed to fix JSON string: Expecting value: line 1 column 1 (char 0)
JSONDecodeError: Expecting ',' delimiter: line 3 column 331 (char 357)
JSONDecodeError: Expecting value: line 1 column 1 (char 0)
Failed to fix JSON string: Expecting value: line 1 column 1 (char 0)
evaluations-qa257.csv saved


100%|███████████████████████████████████████████████████| 15/15 [00:39<00:00,  2.63s/it]


JSONDecodeError: Expecting ',' delimiter: line 3 column 433 (char 461)
JSONDecodeError: Expecting ',' delimiter: line 3 column 356 (char 384)
JSONDecodeError: Expecting ',' delimiter: line 3 column 459 (char 491)
JSONDecodeError: Expecting ',' delimiter: line 3 column 244 (char 272)
JSONDecodeError: Expecting ',' delimiter: line 3 column 224 (char 252)
JSONDecodeError: Expecting ',' delimiter: line 3 column 384 (char 412)
JSONDecodeError: Expecting ',' delimiter: line 3 column 263 (char 289)
evaluations-qa258.csv saved


 53%|███████████████████████████▋                        | 8/15 [00:21<00:18,  2.58s/it]